Strucutre du système : \
-> La classe travailleur qui caractérise un worker, avec sa position, sa zone, la commande et l'article qu'il traite\
-> La classe event qui décrit un évènement avec le numéro du worker en question et l'instant de l'évènement\
-> La classe priority_queue qui implément une file de priorité d'évènements en fonction de l'instant de réalisation, avec les méthodes add() et remove()\
-> La classe waiting_order qui définit les élèments dans le bufffer avec le numéro de commande et le prochain article à récupérer\
-> la classe state avec une liste de workers, une liste de buffers (chaque buffer est une liste de waiting_order) et une file de priorité pour gérer les évènements\
-> La classe order qui définit une commande avec le nombre d'articles nb et la liste d'articles item

Le système est caractérisé par un certains nombres de variables qu'on ne modifiera pas, et une liste de commandes "schedule" à traiter

Un simulation est initialisée avec les bones zones, et les états à zéro : initialize_state(zones), zones étant la liste des zones attribuées aux employés (de taille nombre d'employés).\
Le programme tourne avec la fonction principale run() qui fait appel à 4 fonctions qui effectuent chacune des actions de base d'un employé : poser un colis (drop()), le donner à l'employé suivant (give()), ramasser un colis (pick_order()), aller chercher un objet (pick_item())

In [14]:
from random import randint
from random import expovariate as expo
import statistics
start = 0 #temporaire juste pour le premier lancement du programme
end = 0
import matplotlib.pyplot as plt

In [2]:
# définit un employé de la chaîne
class travailleur:
    def __init__(self, _zone):
        self.pos = -1 #position
        self.order = - 1 #le numéro de la commande en cours, -1 le worker est dispo
        self.item = 0 # le numéro du prochain objet à ramasser
        self.zone = _zone #la zone attribuée à l'employé

In [3]:
# définit un évènement (isntant et worker concerné)
class event:
    def __init__(self, t, _nb_worker):
        self.time = t
        self.worker = _nb_worker

    def switch(self, event2):
        self.time, event2.time = event2.time, self.time
        self.worker, event2.worker = event2.worker, self.worker

#définit les règles d'une chaîne de priorité
class priority_queue:
    def __init__(self):
        self.queue = []
    
    def add(self, event1):
        self.queue.append(event1)
        bool = True
        k = len(self.queue) - 2
        while bool:
            if k<0 :
                bool = False
            elif self.queue[k].time<=self.queue[k+1].time:
                bool = False
            else : 
                self.queue[k].switch(self.queue[k+1])
                k-=1
    
    def remove(self):
        if len(self.queue) == 0:
            print("Error")
        else :
            event1 = self.queue[0]
            self.queue = self.queue[1 ::]
            return event1
        
    def affiche(self) :
        mot = ''
        for k in self.queue :
            mot = mot + str(k.worker) + ' ' + str(int(k.time*10)/10) + '|'
        print(mot)

In [4]:
#définit une commande qui attend dans un buffer
class waiting_order:
    def __init__(self, _order, _item) :
        self.order = _order
        self.item = _item

In [5]:
# définit l'état du système
class state:
    def __init__(self, n, zones):
        self.workers = []
        for k in range(n): #le nombre d'employés
            self.workers.append(travailleur(zones[k]))
        self.buffer=[] # liste des buffers, chaque buffer contient des commandes
        for k in range(n-1):
            self.buffer.append([])
        self.events = priority_queue()
        for k in range(n):
            self.events.add(event(0 , k))

In [6]:
# Une commande
class order:
    default_n = randint(start, end)
    default_items=[]
    for k in range(default_n):
            default_items.append(randint(0,warehouse))

    def __init__(self, n = -1, items = []):
        if n!=-1:
            self.nb = n
            self.item = items
            self.item.sort()
        else : 
            self.n = randint(start, end)
            self.item=[]
            for k in range(self.n):
                self.item.append(randint(0,warehouse))
            self.item.sort()     

In [7]:
# Initialise une liste de commandes
"""length = 1000 #taille de la commande
schedule = []
for k in range(length):
    schedule.append(order())
data=[]
for k in range(length):
    data.append(schedule[k].item)
print(data)"""

'length = 1000 #taille de la commande\nschedule = []\nfor k in range(length):\n    schedule.append(order())\ndata=[]\nfor k in range(length):\n    data.append(schedule[k].item)\nprint(data)'

In [8]:
# Retourne une liste de commandes (toujours la même), de la taille définie au-dessus
def liste_commandes():
    if True : #Stocke la liste de commandes à initialiser dans data
        data = [[1, 5, 31, 36, 52, 58, 99, 106, 112, 140], [4, 34, 41, 46, 57, 59, 61, 64, 71, 108, 116, 142, 145], [12, 16, 19, 46, 136, 147], [6, 33, 53, 58, 65, 80, 86, 86, 97, 122, 147], [3, 15, 64, 74, 74, 94, 105, 121, 141], [4, 27, 71, 72, 102, 118, 134, 148], [10, 30, 55, 92, 109, 143], [3, 29, 31, 32, 50, 61, 79, 103, 121, 131], [44, 56, 76, 93, 103, 106, 142], [0, 35, 40, 66, 70, 145], [14, 26, 30, 44, 47, 74, 96, 98, 109, 127, 140], [37, 49, 53, 85, 96, 119], [4, 44, 46, 76, 85, 86, 93, 120, 146], [9, 10, 47, 47, 90, 93, 95, 106, 120, 127], [35, 56, 64, 75, 84, 100, 101, 102, 108, 132], [11, 34, 40, 77, 89, 121, 130, 139, 145], [13, 45, 46, 50, 53, 59, 63, 65, 88, 92, 109, 125, 138], [11, 32, 42, 45, 47, 79, 93, 93, 130, 148], [1, 26, 48, 101, 108, 131, 138], [14, 34, 47, 64, 83, 91], [19, 20, 25, 31, 51, 68, 83, 100, 105, 120, 143, 144, 144], [7, 13, 30, 63, 74, 81, 89, 97, 98, 113, 115, 148], [9, 19, 21, 23, 43, 43, 48, 57, 87, 124, 126, 131], [27, 32, 61, 65, 66, 75, 85, 126, 138], [4, 10, 38, 40, 51, 72, 74, 104, 107, 117, 144], [8, 24, 26, 41, 48, 48, 49, 59, 72, 97], [14, 33, 60, 64, 65, 81, 87, 87, 108, 118, 132, 148], [14, 27, 57, 63, 87, 93, 95, 103, 111, 122, 124], [21, 36, 42, 78, 92, 98, 107, 126, 150], [18, 24, 38, 44, 49, 85, 98, 104, 118, 122, 137], [10, 21, 33, 37, 43, 61, 64], [3, 45, 54, 58, 59, 62, 71, 83, 123, 123, 145], [29, 60, 93, 112, 120, 137], [7, 23, 54, 60, 78, 80, 90, 103, 119], [2, 38, 38, 39, 53, 67, 118, 128, 129], [16, 26, 39, 39, 42, 44, 44, 67, 68, 119, 123, 133], [10, 13, 19, 22, 34, 38, 48, 53, 75, 110, 118, 148], [18, 19, 51, 63, 66, 69, 82, 111], [21, 43, 79, 89, 104, 105, 110, 125, 133, 137, 150], [1, 46, 51, 57, 92, 92, 111, 114, 116, 122, 124, 147], [1, 15, 21, 42, 44, 78, 90, 94, 129, 141, 149], [8, 29, 32, 33, 49, 66, 77, 108, 135], [16, 29, 31, 32, 50, 84, 107, 109, 123, 128, 131, 140], [4, 11, 21, 36, 43, 62, 71, 78, 78, 104, 105, 135], [30, 35, 63, 63, 81, 82, 85, 86, 128], [17, 55, 56, 68, 80, 82, 131, 142, 150], [23, 24, 50, 81, 94, 99, 102, 140], [8, 12, 12, 56, 86, 86, 101, 130], [5, 18, 25, 31, 40, 43, 90, 113, 130], [4, 47, 69, 89, 91, 111], [10, 11, 12, 76, 83, 87, 89, 90, 116, 118, 149], [3, 9, 15, 16, 30, 36, 67, 82, 90, 91, 125, 126], [19, 29, 47, 52, 53, 56, 67, 75, 76, 82, 130, 140], [9, 23, 62, 62, 79, 116, 119, 123], [30, 44, 71, 71, 79, 90, 100, 123, 145], [1, 7, 57, 111, 141, 149], [36, 42, 43, 70, 83, 115, 129, 132], [19, 43, 65, 96, 132, 148, 149], [21, 61, 61, 100, 107, 112, 116, 121], [1, 39, 57, 61, 63, 146, 150], [50, 58, 66, 78, 95, 105, 125, 125, 128, 129, 150], [22, 23, 28, 58, 64, 70, 85, 91, 102, 113, 117, 119, 148], [0, 26, 31, 53, 57, 71, 76, 88, 128, 130, 141, 147, 148], [4, 13, 33, 114, 119, 140], [0, 7, 39, 55, 59, 67, 103, 110, 110, 120, 122, 125, 134], [0, 1, 3, 21, 39, 74, 83, 83, 99, 121, 132, 135, 149], [16, 23, 43, 112, 130, 144], [6, 28, 50, 125, 133, 134], [3, 7, 12, 53, 116, 122, 134], [5, 18, 40, 82, 95, 95, 97, 122, 130, 134, 140, 150], [3, 4, 5, 60, 66, 69, 74, 97, 138, 143], [5, 23, 52, 53, 57, 113, 125, 147], [20, 47, 74, 82, 113, 114, 146, 149], [30, 74, 85, 112, 124, 132, 137], [2, 19, 36, 69, 76, 79, 95, 116, 121, 133], [12, 16, 41, 56, 63, 77, 118], [2, 21, 34, 46, 70, 74, 111], [10, 38, 60, 79, 82, 85, 118, 125], [13, 28, 29, 34, 35, 46, 50, 81, 95, 112, 119, 138], [1, 5, 90, 111, 119, 143, 146], [22, 28, 60, 88, 90, 111, 133, 138], [3, 10, 24, 74, 77, 77, 107, 107], [13, 16, 26, 52, 86, 135, 139], [1, 13, 28, 42, 59, 65, 71, 82, 105, 110, 139], [1, 33, 36, 79, 92, 113, 133, 133, 142, 149], [13, 36, 79, 90, 94, 105], [3, 9, 18, 20, 35, 55, 76, 84, 90, 101, 119, 126], [4, 6, 6, 33, 46, 110, 139, 142, 148], [9, 12, 57, 69, 78, 102, 107], [0, 13, 15, 19, 28, 43, 60, 119, 135, 140], [5, 20, 33, 40, 56, 64, 80, 86, 91, 134, 139, 144], [11, 32, 79, 81, 85, 129, 132, 134, 136], [7, 44, 97, 101, 131, 140], [17, 25, 37, 40, 41, 81, 100, 115, 119, 120], [24, 63, 69, 74, 110, 129, 148], [19, 58, 65, 65, 65, 86, 106, 107, 115, 118, 128, 133, 139], [47, 48, 56, 58, 66, 77, 131], [25, 31, 41, 63, 64, 77, 77, 80, 93, 118], [1, 4, 18, 29, 38, 44, 63, 68, 70, 149], [55, 87, 103, 103, 105, 130], [1, 25, 67, 79, 99, 106, 120, 149, 150], [10, 10, 23, 48, 63, 80, 80, 88, 90, 97, 102, 116], [3, 17, 43, 49, 51, 67, 94], [71, 83, 85, 96, 99, 99, 102, 118], [3, 39, 54, 64, 73, 89, 97, 135], [21, 39, 46, 82, 89, 91, 119, 122, 124, 148], [31, 33, 33, 39, 52, 55, 55, 60, 60, 80, 139, 150], [0, 13, 35, 38, 67, 83, 100, 114, 125, 138], [6, 12, 34, 38, 58, 63, 70, 93, 120, 145], [0, 8, 56, 75, 80, 82], [7, 18, 28, 31, 47, 64, 91, 99, 111, 132, 136], [5, 5, 8, 20, 22, 25, 48, 79, 79, 79, 98, 118, 121], [3, 43, 50, 53, 54, 57, 84, 86, 113, 125, 134], [37, 57, 66, 68, 117, 122, 122, 143], [8, 8, 60, 76, 98, 101, 145], [8, 8, 39, 100, 103, 119, 135, 150], [3, 23, 39, 45, 69, 118, 120], [4, 7, 14, 19, 24, 54, 69, 100, 106, 133], [4, 18, 23, 38, 68, 83, 106, 123, 123, 127, 138, 150], [29, 33, 37, 53, 55, 60, 68, 88, 91, 95, 110, 123], [22, 29, 36, 41, 44, 48, 84, 87, 99, 130, 133], [0, 4, 17, 37, 69, 89, 98, 102, 105, 117, 133], [3, 9, 35, 57, 82, 91, 109, 116, 118, 134, 139, 142, 145], [4, 15, 38, 39, 49, 51, 54, 62, 71, 91, 134, 135, 147], [8, 22, 63, 102, 134, 141, 144, 146], [1, 23, 31, 56, 76, 121, 137], [15, 17, 22, 36, 48, 81, 103, 104, 114, 123, 129, 132, 136], [20, 24, 61, 77, 89, 118, 137, 140, 141], [13, 26, 43, 65, 71, 98, 101, 111, 118, 123, 133, 142], [3, 5, 9, 15, 21, 40, 49, 52, 62, 63, 65, 94, 96], [56, 59, 74, 78, 91, 95, 101, 107, 149], [5, 5, 19, 19, 58, 61, 68, 76, 78, 86, 123, 130, 147], [3, 29, 32, 50, 57, 62, 67, 77, 83, 86, 119, 139, 148], [5, 6, 8, 10, 11, 17, 20, 30, 69, 100, 145, 149, 150], [5, 21, 83, 85, 92, 145], [5, 34, 49, 70, 83, 109, 109, 117, 120, 121, 125, 128], [5, 6, 10, 13, 29, 35, 92, 95, 122, 125, 140, 141], [42, 45, 50, 71, 75, 84, 123], [15, 35, 47, 78, 91, 116, 132], [3, 30, 30, 42, 44, 49, 80, 108, 115, 126, 128, 140], [1, 68, 79, 86, 92, 92, 131], [50, 74, 78, 84, 129, 150], [35, 41, 53, 64, 68, 99, 132, 142, 144], [14, 24, 24, 28, 33, 54, 58, 62, 75, 82, 83, 83, 119], [43, 56, 70, 81, 104, 109, 111, 138], [3, 14, 61, 69, 89, 92, 110, 113, 116, 126, 139], [31, 42, 56, 92, 106, 119], [17, 24, 26, 31, 96, 119, 136, 149], [7, 41, 73, 85, 85, 100, 100, 106, 131, 134, 138], [6, 49, 59, 59, 110, 113, 136], [11, 23, 25, 44, 52, 52, 74, 74, 146], [25, 38, 61, 61, 67, 77, 130, 140], [8, 11, 15, 44, 74, 91, 99, 101, 118, 120, 144, 147], [7, 12, 16, 48, 62, 67, 72, 81, 99, 100, 109, 122, 139], [56, 63, 77, 84, 89, 102, 103, 105, 111, 111, 134, 143], [39, 42, 101, 104, 107, 139], [5, 35, 54, 99, 99, 102, 109], [1, 5, 23, 56, 93, 113, 147], [31, 44, 68, 99, 107, 126, 148], [21, 26, 47, 54, 58, 72, 76, 115, 149], [20, 21, 29, 36, 36, 67, 80, 85, 105, 113, 140, 144], [5, 20, 35, 36, 56, 75, 102, 107, 141, 142, 142], [15, 21, 22, 32, 59, 82], [35, 49, 53, 57, 74, 84, 112, 123], [16, 18, 28, 59, 81, 91, 92, 96, 97, 105, 146, 146], [18, 27, 39, 48, 60, 67, 93, 101, 102, 120, 122, 123, 129], [24, 26, 33, 37, 56, 62, 64, 83, 105, 113, 133, 136, 144], [24, 24, 34, 37, 41, 48, 70, 73, 76, 80, 102, 106, 132], [22, 42, 58, 61, 98, 101, 111, 150], [7, 12, 19, 79, 103, 139, 146], [36, 69, 86, 97, 100, 140, 145], [45, 104, 111, 112, 124, 133, 137], [12, 21, 26, 30, 43, 121, 138, 144], [91, 97, 102, 110, 123, 143, 145], [16, 17, 51, 96, 122, 131, 145, 148], [11, 31, 54, 65, 70, 74, 82, 88, 98, 104, 120, 130], [2, 28, 34, 73, 76, 116, 127, 129, 132, 141, 149], [14, 33, 45, 66, 92, 94, 102, 134, 137, 147], [4, 24, 25, 39, 53, 66, 73, 80, 87, 91, 107, 118, 127], [20, 57, 62, 100, 102, 109, 144], [6, 19, 39, 67, 74, 84, 101, 122, 129, 130, 131, 133, 137], [11, 43, 63, 63, 97, 99, 100, 103, 128, 132, 141], [8, 10, 16, 32, 46, 50, 79, 87, 132, 140, 142, 144], [6, 19, 40, 62, 75, 75, 109, 115], [30, 63, 71, 75, 79, 93, 95, 105, 110, 122, 135], [25, 26, 79, 116, 148, 148], [4, 7, 7, 13, 31, 60, 76, 93, 106, 116, 150], [9, 21, 50, 51, 66, 82, 94, 101, 127], [41, 43, 55, 88, 118, 139, 139], [7, 10, 44, 45, 65, 65, 66, 69, 98, 101, 113, 119, 139], [7, 43, 55, 61, 68, 84, 93, 112, 123, 127, 133, 134, 149], [17, 21, 35, 37, 69, 83, 89, 101, 111, 126], [3, 24, 47, 70, 79, 114, 144], [10, 10, 15, 39, 39, 48, 81, 87, 101, 129, 139], [6, 28, 31, 66, 100, 109, 115, 119, 127, 140, 145], [11, 24, 31, 42, 46, 55, 91, 102, 124], [4, 5, 10, 18, 91, 94], [23, 35, 64, 64, 86, 90, 96, 105, 107, 109, 137], [8, 20, 28, 30, 76, 81, 84, 110, 112, 127, 138, 139, 149], [11, 12, 28, 45, 67, 72, 86, 87, 92, 93, 98, 106], [8, 61, 101, 111, 125, 126, 131], [14, 15, 16, 34, 38, 75, 79, 98, 119, 131, 132, 135], [52, 60, 63, 107, 133, 137], [6, 28, 40, 42, 57, 67, 75, 116, 139, 140, 145], [35, 56, 72, 78, 87, 117, 129, 133, 135], [3, 11, 15, 30, 33, 73, 77, 78, 82, 101], [7, 15, 27, 29, 38, 58, 90, 96, 119, 126, 127, 128, 147], [2, 11, 12, 43, 71, 86, 108, 114], [10, 35, 59, 62, 78, 80, 87, 94, 102, 132, 147, 149], [12, 28, 40, 42, 81, 85, 95, 132, 134, 141, 143, 147], [41, 58, 62, 65, 70, 80, 98, 124, 136], [65, 65, 72, 105, 117, 121, 145], [10, 21, 25, 49, 77, 78, 80, 94, 108, 108, 134], [47, 53, 71, 72, 79, 79, 90, 112, 113, 115, 125, 126, 150], [15, 30, 31, 33, 121, 140], [30, 54, 64, 67, 84, 126, 148], [3, 16, 43, 48, 48, 91, 91, 95, 124, 146], [29, 51, 80, 98, 108, 115, 119, 136], [29, 39, 43, 45, 52, 62, 71, 76, 102, 106, 107, 149], [40, 51, 61, 62, 112, 127, 132, 145], [54, 77, 85, 100, 111, 140], [31, 37, 59, 78, 89, 122], [9, 10, 22, 25, 48, 148], [56, 68, 81, 97, 102, 115, 125, 137], [9, 9, 11, 41, 49, 59, 94, 130, 149], [18, 69, 70, 105, 128, 132], [17, 34, 35, 69, 79, 85, 93, 94, 101, 112, 133, 133], [5, 24, 68, 80, 111, 123, 128], [7, 12, 15, 23, 49, 67, 85, 95, 100, 110], [2, 8, 43, 121, 123, 124, 125, 144, 145, 147], [15, 17, 26, 30, 45, 51, 60, 62, 80, 93, 101], [8, 10, 35, 38, 55, 64, 65, 85, 94, 146], [3, 11, 35, 45, 49, 110], [35, 38, 48, 52, 57, 100], [2, 22, 32, 62, 100, 105, 110, 112, 120, 146], [3, 4, 8, 17, 24, 45, 51, 76, 117, 139, 140], [56, 110, 111, 114, 127, 138], [5, 14, 35, 98, 120, 148], [13, 49, 54, 66, 88, 104, 133, 135, 147], [1, 1, 7, 84, 110, 121, 125, 150], [6, 31, 32, 57, 68, 110, 145], [8, 15, 19, 35, 36, 38, 82, 94, 110, 114, 134, 136, 145], [1, 12, 18, 27, 36, 53, 77, 93, 131, 138], [6, 7, 12, 24, 53, 69, 92, 105], [36, 41, 51, 57, 75, 76, 98, 112, 117, 120, 121], [25, 48, 66, 76, 104, 120], [44, 54, 55, 99, 138, 150], [6, 6, 14, 20, 27, 80, 85, 86, 88, 139], [23, 28, 33, 47, 52, 53, 62, 82, 89, 95, 102, 146], [1, 6, 13, 64, 80, 84, 97, 97, 100, 108, 127], [5, 5, 21, 31, 51, 64, 64, 68, 77, 85, 106, 108, 142], [4, 4, 25, 34, 35, 60, 61, 87, 105, 128], [29, 30, 70, 84, 91, 101, 107, 122, 136, 142, 148], [19, 25, 41, 44, 48, 69, 97, 100, 102, 116, 136], [9, 12, 35, 43, 53, 64, 67, 133, 143], [4, 5, 17, 43, 64, 88, 113, 136, 139], [6, 15, 28, 44, 54, 57, 82, 92, 108, 125, 137, 140, 140], [17, 23, 34, 57, 60, 68, 90, 95, 103, 142, 150], [20, 29, 39, 40, 53, 70, 78, 102, 110, 113, 133], [9, 58, 65, 78, 84, 87, 89, 95, 101, 121, 133, 144, 149], [4, 34, 41, 64, 84, 112, 114, 131], [18, 37, 42, 77, 78, 79, 90, 100, 104, 110, 113, 120], [27, 50, 93, 105, 108, 134, 148], [18, 26, 43, 44, 58, 100, 107, 132], [13, 17, 25, 41, 68, 70, 92, 105, 115, 119, 129, 134], [3, 19, 25, 35, 42, 42, 45, 54, 55, 79, 146], [20, 32, 46, 53, 57, 62, 74, 82, 86, 89, 103, 122, 150], [6, 11, 14, 34, 75, 77, 80, 82, 85, 102, 120, 133, 138], [29, 65, 81, 86, 91, 112, 115, 129, 134], [4, 30, 33, 52, 72, 73, 95, 107, 130, 140], [15, 24, 39, 43, 60, 75, 91, 95, 108, 127, 129], [6, 17, 53, 64, 72, 89, 103], [1, 10, 54, 56, 61, 73, 73, 109], [4, 4, 45, 102, 116, 121, 122, 146], [14, 16, 20, 43, 57, 92, 104, 147, 150], [6, 18, 19, 29, 64, 96, 99, 104, 138], [7, 14, 58, 90, 100, 118, 120, 132], [3, 6, 30, 34, 39, 43, 50, 74], [0, 3, 14, 19, 23, 28, 59, 78, 109, 113, 120], [9, 11, 23, 38, 51, 58, 67, 91, 93, 96, 98, 115, 130], [33, 39, 73, 73, 84, 96, 117, 124, 126], [0, 16, 28, 49, 120, 123, 125, 141, 143], [39, 41, 72, 97, 129, 146, 146], [5, 13, 18, 23, 37, 54, 59, 79, 93, 108, 113, 129, 144], [0, 2, 12, 54, 117, 124], [18, 19, 27, 30, 66, 125, 125, 136, 146], [21, 47, 48, 70, 75, 90, 92, 124], [60, 62, 71, 77, 100, 111, 114, 147], [0, 3, 6, 51, 72, 131, 134], [7, 19, 45, 47, 56, 72, 75, 83, 111, 142, 149], [31, 33, 33, 54, 61, 70, 73, 80, 86, 150], [25, 34, 54, 62, 98, 124, 130, 130], [6, 7, 38, 55, 68, 84, 89, 99, 116, 149], [19, 41, 81, 83, 89, 95, 106, 141], [20, 21, 23, 41, 48, 59, 69, 79, 96, 105, 145, 150], [58, 61, 79, 94, 119, 126, 146], [4, 15, 28, 67, 93, 131, 131, 137, 145], [32, 44, 48, 93, 94, 110, 122, 143], [14, 37, 55, 74, 107, 124, 132, 132], [18, 19, 54, 62, 66, 93, 110, 113, 129, 131, 133, 140, 146], [8, 16, 18, 21, 35, 76, 130], [5, 17, 36, 41, 44, 81, 89, 92, 99, 111, 114, 129, 144], [32, 76, 112, 116, 118, 137], [12, 28, 39, 41, 41, 51, 102, 104, 106], [21, 43, 45, 65, 80, 85, 89, 90, 132], [2, 6, 33, 88, 123, 130, 133], [31, 38, 40, 40, 77, 78, 82, 143, 148], [3, 29, 39, 59, 60, 131, 135], [0, 0, 12, 46, 101, 118], [23, 31, 38, 63, 68, 77, 79, 80, 81, 99, 110, 113, 134], [1, 37, 44, 65, 78, 102], [4, 8, 59, 76, 109, 130], [14, 28, 52, 53, 76, 128, 136, 145, 150], [5, 10, 25, 29, 32, 43, 48, 55, 67, 107, 117, 117], [15, 23, 54, 74, 104, 119], [15, 16, 22, 26, 43, 44, 63, 81, 82, 113, 120, 143], [7, 26, 35, 43, 64, 77, 87, 97, 102, 122, 122, 131, 133], [35, 85, 112, 121, 125, 129, 140, 144, 146], [17, 29, 51, 62, 70, 83, 96], [18, 18, 35, 70, 82, 88, 127, 143], [6, 12, 22, 26, 31, 32, 40, 65, 86, 107, 129, 135], [1, 3, 33, 38, 54, 64, 73, 134], [3, 14, 15, 19, 39, 40, 55, 59, 63, 77, 127, 148], [41, 49, 51, 80, 90, 121], [0, 16, 42, 48, 113, 118, 137, 140, 147], [2, 10, 30, 37, 39, 44, 44, 64, 68, 70, 73, 141], [13, 22, 32, 64, 84, 107, 115, 118, 136], [3, 3, 25, 45, 46, 57, 82, 86, 98, 98, 101, 104, 116], [9, 18, 24, 40, 47, 72, 103, 114, 121], [34, 63, 104, 105, 107, 143, 144], [23, 35, 44, 82, 92, 95, 128, 129, 134, 136, 147, 147], [14, 15, 23, 108, 123, 127, 147], [7, 38, 39, 40, 45, 63, 65, 97, 146], [23, 29, 34, 41, 66, 117], [24, 43, 98, 106, 130, 130, 132, 143], [4, 19, 21, 34, 46, 47, 61, 105, 139], [4, 24, 36, 102, 114, 119], [4, 6, 53, 73, 74, 99, 115, 122, 129, 132, 136, 137, 149], [7, 53, 60, 62, 73, 76, 88, 97, 106, 130], [19, 21, 28, 56, 58, 60, 108, 115, 116, 130, 140, 147], [61, 86, 87, 88, 90, 110, 114, 131, 146, 146], [15, 16, 24, 44, 45, 59, 64, 85, 86, 95, 103, 117], [6, 21, 21, 47, 61, 68, 71, 79, 80, 90, 95, 100, 122], [0, 5, 24, 83, 102, 103, 123, 126], [30, 50, 57, 102, 117, 120, 127, 145], [15, 23, 29, 36, 67, 72, 75, 79, 91, 122, 124, 128], [5, 21, 48, 81, 87, 97, 105, 128, 137], [46, 47, 53, 56, 84, 89, 93, 105, 133, 143], [0, 1, 13, 38, 53, 74, 96, 114, 119, 119, 121, 128, 133], [10, 29, 44, 47, 55, 67, 72, 73, 87, 109, 124], [10, 33, 66, 86, 108, 134], [44, 67, 104, 110, 111, 113, 138, 144], [1, 5, 15, 45, 61, 64, 80, 101, 126, 135], [11, 30, 63, 64, 75, 81, 90, 93, 112, 117, 129, 147], [27, 40, 46, 124, 128, 149], [1, 11, 36, 44, 74, 105, 113, 130, 141, 150], [4, 31, 45, 58, 60, 94, 99, 103, 110, 129, 137, 141], [17, 24, 27, 28, 35, 60, 73, 82, 99, 108, 136, 139], [0, 44, 55, 66, 84, 91, 112, 131, 148], [5, 6, 41, 42, 56, 56, 72, 78, 81, 83, 103, 106, 136], [18, 21, 26, 28, 48, 66, 78, 132, 134], [14, 18, 21, 27, 40, 87, 103, 119, 123, 134, 145, 147], [13, 14, 35, 49, 74, 128, 145], [15, 34, 99, 125, 125, 140, 148], [5, 6, 20, 37, 98, 100, 103, 104, 124, 124, 131, 133, 143], [7, 37, 66, 83, 97, 108, 115, 119], [14, 16, 30, 70, 76, 86, 93, 111, 135, 143, 147], [17, 22, 25, 52, 76, 82, 118, 131, 150], [5, 34, 34, 51, 58, 71, 82, 98], [14, 14, 42, 69, 75, 78, 98], [12, 20, 42, 54, 56, 57, 77, 83, 86, 100, 115, 126], [39, 41, 50, 53, 57, 70, 83, 113, 142, 145, 146, 147, 148], [9, 18, 33, 39, 56, 61, 61, 67, 116, 124, 148], [38, 39, 68, 101, 102, 146, 149], [1, 33, 47, 49, 51, 70, 90, 100, 110, 113, 126], [34, 52, 57, 66, 76, 84, 95, 97, 111, 111, 112, 131], [2, 2, 3, 11, 18, 43, 58, 97, 118, 120], [39, 51, 72, 85, 116, 136], [21, 32, 45, 57, 63, 68, 74, 92, 94, 102, 103, 103, 143], [50, 58, 71, 75, 80, 84], [19, 30, 62, 74, 74, 107, 115, 135], [8, 10, 17, 47, 63, 66, 73, 86, 90, 91, 114, 119, 148], [2, 43, 71, 72, 80, 81, 85, 86, 124], [0, 16, 23, 32, 42, 43, 57, 87, 113, 116, 129, 136], [16, 23, 46, 81, 81, 103, 148], [14, 21, 23, 26, 33, 44, 57, 66, 66, 71, 98, 102, 143], [0, 0, 9, 64, 105, 125, 146], [30, 33, 42, 50, 51, 55, 59, 60, 103, 110, 122, 127, 128], [5, 7, 10, 50, 82, 97, 100, 113, 122, 125, 137, 147], [10, 10, 29, 31, 59, 67, 70, 72, 82, 98, 99, 131, 141], [1, 15, 38, 47, 72, 86, 96, 126], [14, 32, 52, 58, 71, 82, 110, 133], [35, 51, 60, 67, 78, 118, 127], [9, 10, 82, 99, 115, 146], [34, 35, 56, 80, 87, 110, 142], [10, 36, 48, 56, 78, 80, 82, 100, 127, 132, 139], [23, 43, 49, 49, 52, 55, 66, 66, 123], [4, 10, 12, 14, 29, 48, 83, 89, 100, 106, 122, 147], [0, 8, 8, 36, 39, 150], [26, 50, 71, 94, 94, 104, 116, 122, 140, 143], [2, 21, 21, 29, 34, 36, 47, 49], [0, 6, 11, 23, 35, 72, 76, 77, 110, 131], [15, 39, 52, 55, 79, 94, 110, 130, 131, 133], [24, 25, 42, 45, 76, 83, 87, 101, 117, 134, 134, 147], [16, 32, 49, 64, 65, 74, 75, 88, 119, 131, 131, 150], [22, 26, 48, 71, 84, 147], [2, 6, 24, 39, 44, 60, 98, 136, 147], [1, 25, 35, 63, 86, 91, 93, 110, 128, 136, 140, 148], [23, 40, 53, 72, 96, 108, 115, 117, 130, 133], [13, 42, 51, 73, 84, 113, 125, 142], [3, 10, 24, 42, 82, 92, 100, 139], [6, 53, 57, 96, 112, 118, 119, 125], [8, 23, 26, 45, 66, 96, 104, 134], [19, 35, 41, 83, 89, 94, 101], [15, 32, 72, 82, 85, 86, 88, 95, 110, 115, 138], [29, 35, 40, 76, 87, 108, 142], [0, 1, 9, 13, 25, 45, 48, 115, 117, 142], [29, 53, 59, 90, 106, 117, 123, 131], [7, 27, 34, 35, 52, 68, 102, 118], [43, 56, 95, 98, 118, 123, 125, 128, 128, 139], [8, 15, 20, 23, 23, 24, 29, 44, 96, 111, 111, 127, 150], [3, 15, 20, 47, 75, 95, 111, 119], [16, 46, 54, 54, 62, 73, 77, 137], [40, 75, 81, 95, 113, 124], [10, 44, 49, 84, 89, 90, 106, 111, 114], [45, 49, 53, 63, 63, 89, 105, 111, 118, 147], [2, 38, 40, 40, 72, 100, 106, 121, 126], [20, 27, 52, 58, 62, 62, 86, 99, 110, 116, 120, 138], [12, 18, 39, 58, 61, 71, 83, 143], [16, 18, 34, 43, 59, 134, 138, 139], [14, 32, 64, 70, 75, 78, 91, 125, 133], [4, 29, 42, 62, 63, 71, 91, 92, 107, 138], [27, 41, 69, 99, 125, 144], [10, 72, 75, 90, 125, 129, 142], [2, 8, 11, 36, 74, 98, 106, 107, 124, 126, 136, 146], [1, 2, 14, 43, 48, 54, 64, 66, 144, 146], [1, 10, 17, 28, 40, 48, 51, 51, 80, 112], [6, 19, 69, 92, 110, 124], [29, 70, 84, 108, 121, 121, 142, 146], [2, 2, 35, 59, 75, 83, 89, 91, 99, 104, 105, 143, 147], [2, 25, 31, 60, 64, 72, 73, 87, 120, 130], [46, 52, 61, 73, 87, 98, 127], [23, 23, 35, 42, 87, 121], [21, 38, 42, 51, 51, 72, 73, 93, 102, 139, 140, 148], [2, 4, 19, 31, 50, 128, 130], [13, 27, 32, 38, 51, 60, 65, 84, 102, 135, 148, 150], [5, 36, 40, 42, 53, 55, 133], [10, 20, 29, 45, 59, 60, 67, 97, 98, 124, 130], [16, 18, 58, 59, 94, 102, 122, 134], [27, 27, 53, 56, 78, 86, 91, 105, 115, 125, 132, 144, 149], [3, 6, 35, 79, 93, 116, 129, 136, 136, 147], [11, 49, 66, 72, 77, 85, 96, 110], [18, 28, 70, 76, 79, 84, 86, 96, 140, 140], [19, 19, 25, 58, 67, 74, 127, 131, 144, 148], [5, 23, 29, 36, 48, 52, 74, 92, 100, 119, 130, 134], [5, 41, 60, 62, 69, 71, 102, 116, 120, 133, 139, 149], [32, 54, 56, 61, 68, 81, 81, 140, 147], [1, 12, 16, 16, 42, 51, 52, 56, 81, 87, 97, 142], [6, 25, 40, 52, 75, 91, 95], [15, 38, 49, 50, 51, 64, 120], [17, 30, 31, 75, 78, 82, 86, 94, 128, 141, 149, 150], [11, 32, 73, 90, 92, 94, 113, 129, 144], [13, 15, 32, 66, 72, 73, 85, 107, 122, 132, 147], [15, 17, 24, 35, 69, 84, 95, 112, 117, 146], [7, 49, 53, 55, 57, 57, 93, 120, 122, 127, 148], [6, 12, 28, 69, 76, 88, 96, 98, 113, 121, 126, 149], [43, 60, 66, 71, 78, 84, 90, 93, 98, 106, 122, 132], [1, 18, 34, 45, 58, 92, 108, 118, 118], [27, 35, 47, 47, 63, 66, 83, 91, 105, 107, 120, 127, 139], [10, 15, 17, 42, 62, 83, 100, 141, 142], [19, 20, 34, 41, 54, 71, 75, 79, 108, 108, 112, 148], [15, 40, 44, 48, 115, 128], [41, 45, 45, 58, 67, 92, 105, 106, 116, 116, 120, 132, 136], [5, 64, 77, 83, 110, 126], [14, 29, 50, 61, 71, 72, 73, 86, 96, 117, 119], [0, 19, 45, 53, 72, 101, 103, 106, 148], [24, 36, 37, 49, 74, 77, 79, 80, 86, 89, 144], [1, 67, 67, 85, 85, 150, 150], [32, 36, 46, 55, 109, 112, 125, 146], [7, 16, 26, 50, 57, 72, 91, 115, 125, 126], [0, 3, 27, 50, 54, 64, 86, 94, 98, 122, 123, 144], [0, 5, 9, 14, 49, 80, 88, 136], [9, 39, 44, 97, 103, 105, 111, 119, 121, 146, 149], [31, 46, 75, 91, 118, 129, 149], [5, 20, 29, 43, 52, 60, 67, 79, 106, 108, 117, 135, 143], [21, 34, 61, 88, 129, 130, 134, 135, 144], [16, 32, 58, 84, 97, 101, 116, 117, 121, 124, 132, 146], [1, 8, 14, 44, 44, 45, 46, 52, 67, 82, 89, 100, 129], [11, 14, 40, 50, 72, 94, 131, 136, 137], [4, 20, 39, 62, 92, 115, 116, 127, 127, 148], [15, 46, 63, 73, 89, 111, 114, 116, 131, 135], [2, 8, 42, 51, 71, 76, 81, 125, 131, 145], [1, 16, 20, 24, 25, 30, 53, 70, 99, 113, 118], [6, 25, 27, 40, 65, 121, 130, 139], [12, 54, 70, 94, 101, 135], [34, 58, 66, 101, 108, 113, 126, 134, 137, 139, 147, 149], [0, 31, 38, 38, 54, 57, 98, 144, 146], [4, 15, 69, 84, 115, 128], [27, 66, 69, 69, 80, 115], [5, 10, 25, 28, 48, 84, 93, 93, 97, 114, 124], [12, 13, 16, 36, 44, 52, 77, 82, 122, 135, 136], [36, 49, 74, 75, 76, 82, 144], [3, 4, 41, 58, 83, 85, 92, 104, 114], [9, 15, 21, 31, 64, 67, 97, 112, 123, 133, 146], [2, 3, 25, 34, 45, 72, 79, 96, 104, 105, 127, 140, 145], [0, 29, 30, 78, 83, 86, 104], [1, 4, 17, 18, 42, 58], [38, 49, 70, 74, 92, 126], [6, 29, 40, 50, 76, 87], [0, 10, 20, 23, 43, 49, 59, 78, 87, 94, 145], [1, 2, 47, 62, 70, 86, 101], [7, 8, 12, 36, 52, 62, 119], [17, 60, 63, 88, 98, 106, 119, 149], [19, 29, 39, 67, 84, 127, 129, 146], [10, 10, 12, 14, 21, 34, 38, 78, 81, 112, 115], [0, 18, 20, 42, 42, 64, 75, 80, 99, 123, 140], [3, 14, 15, 53, 86, 97, 98, 123], [15, 18, 35, 81, 83, 107, 125, 127, 147], [51, 62, 94, 108, 114, 126, 129, 132, 149], [6, 63, 74, 90, 121, 131, 145], [4, 34, 67, 71, 109, 133, 146], [46, 48, 84, 85, 118, 121, 141, 142], [10, 10, 40, 41, 43, 49, 67, 127, 136], [7, 44, 45, 54, 58, 62, 67, 68, 70, 73, 108, 130, 144], [12, 17, 50, 56, 70, 77, 101, 128, 130], [19, 39, 102, 108, 114, 138, 145], [40, 41, 86, 93, 98, 103, 115], [8, 10, 13, 41, 82, 83, 102], [76, 77, 102, 119, 127, 140], [10, 30, 32, 41, 43, 44, 46, 95, 139, 140, 142], [18, 23, 40, 40, 49, 108, 119, 125, 129, 139, 140, 148], [5, 19, 30, 56, 64, 112], [0, 2, 48, 57, 62, 90, 115, 126, 127, 131, 134, 140], [15, 33, 65, 77, 84, 89, 118, 141, 141], [13, 26, 32, 46, 55, 85, 94, 97, 117, 125], [17, 23, 24, 36, 41, 53, 55, 100, 112, 120, 120, 130], [27, 58, 64, 65, 94, 110, 114, 117, 122, 126, 132, 133, 143], [10, 14, 41, 62, 78, 78, 100, 115, 122, 130, 132, 142, 144], [17, 26, 30, 53, 61, 83, 137, 140], [13, 28, 46, 52, 53, 61, 77, 81, 102, 114, 119], [3, 51, 55, 88, 107, 107, 109, 142, 147], [15, 24, 79, 93, 107, 132, 147], [15, 15, 22, 62, 79, 80, 89, 93, 97, 123, 141], [19, 20, 27, 35, 38, 41, 82, 99, 104, 106, 110, 133, 146], [2, 8, 15, 56, 60, 87, 89, 116, 121, 128, 131, 136, 147], [34, 35, 51, 52, 57, 76, 126], [0, 12, 20, 24, 32, 40, 49, 80, 100, 108, 117, 128, 143], [2, 19, 19, 28, 92, 99, 129, 133, 143, 146], [1, 10, 57, 87, 107, 133], [39, 39, 42, 60, 108, 147], [6, 37, 54, 85, 113, 121], [30, 56, 81, 83, 111, 131, 133], [4, 10, 26, 37, 56, 83, 90, 100, 108, 109, 113, 141, 144], [6, 20, 22, 29, 40, 40, 85, 122, 127, 137], [8, 20, 82, 83, 85, 89, 103, 133, 140], [22, 67, 96, 116, 117, 131], [13, 17, 45, 47, 85, 87, 97, 112, 148], [1, 2, 18, 39, 41, 47, 89, 91, 102, 107, 128], [40, 62, 100, 107, 140, 147], [2, 8, 8, 25, 71, 77, 99, 102], [6, 6, 20, 52, 87, 88, 97, 98, 117, 117, 118, 131], [12, 38, 54, 65, 77, 88, 89, 95, 130, 134, 143, 150], [29, 30, 41, 57, 74, 76, 94, 131], [1, 15, 38, 52, 77, 126], [11, 21, 55, 63, 79, 83, 84, 102, 108, 121, 134, 146], [0, 18, 46, 92, 94, 101, 104, 114, 125, 136], [3, 11, 31, 31, 31, 57, 62, 66, 78, 90, 93, 139], [11, 35, 59, 67, 75, 89, 133], [5, 16, 19, 25, 52, 102, 106, 139], [1, 3, 27, 37, 57, 114, 127, 132, 143], [7, 11, 41, 86, 87, 95, 100, 105, 107, 108], [4, 15, 24, 52, 62, 66, 68, 96, 97, 114, 132, 141], [3, 5, 20, 114, 116, 120, 126, 148], [8, 13, 17, 20, 24, 33, 43, 49, 59, 68, 91, 121, 130], [57, 59, 93, 95, 95, 105, 109, 142, 143], [19, 44, 46, 47, 56, 125, 137, 137, 141, 141], [2, 8, 36, 40, 41, 46, 61, 121, 132], [24, 38, 56, 58, 111, 128], [25, 31, 80, 81, 115, 130], [9, 11, 48, 50, 64, 67, 81, 81, 92, 106, 106], [2, 10, 19, 32, 42, 105, 123], [27, 39, 58, 77, 103, 104, 133], [0, 10, 37, 48, 55, 57, 68, 79, 95, 99, 113, 131], [6, 11, 30, 41, 60, 64, 74], [23, 46, 56, 57, 89, 110, 142], [28, 57, 65, 82, 92, 101], [11, 11, 15, 27, 44, 82, 84, 85, 91, 116, 125, 128], [32, 47, 102, 108, 121, 132, 137, 144], [0, 11, 16, 50, 58, 81, 102, 115, 134, 135, 141, 149], [30, 84, 94, 109, 112, 121, 134, 135], [8, 10, 15, 36, 64, 69, 129, 130], [32, 36, 39, 54, 59, 61, 63, 64, 102, 105, 129, 145], [19, 72, 86, 105, 116, 130, 136, 139, 147], [31, 40, 42, 45, 57, 65, 65, 66, 84, 85, 96, 105, 146], [11, 28, 68, 82, 91, 121, 139, 139], [11, 21, 21, 30, 31, 44, 118, 122, 137], [16, 39, 42, 43, 43, 97, 99, 99, 103, 106, 130, 139], [2, 16, 64, 100, 121, 142], [12, 23, 38, 65, 73, 102, 120, 127, 127, 129], [54, 79, 95, 100, 113, 132, 135, 144], [5, 48, 51, 60, 61, 91, 136], [7, 10, 44, 70, 76, 78, 87, 106, 123, 124], [1, 59, 61, 61, 78, 82, 94, 96, 135], [3, 15, 25, 38, 67, 72, 128, 133, 142], [8, 11, 31, 35, 36, 67], [4, 15, 51, 64, 85, 85, 107, 108, 111, 150], [5, 8, 22, 24, 120, 135, 144], [0, 16, 41, 47, 58, 59, 78, 102, 104, 106, 124, 135], [14, 32, 33, 44, 56, 64, 91, 105, 106, 126, 137, 141, 146], [7, 23, 37, 56, 64, 79, 80, 83, 102, 109, 115, 129, 143], [18, 32, 48, 49, 50, 79, 79, 87, 88, 122, 122], [5, 5, 14, 24, 25, 26, 52, 52, 54, 84, 94, 113], [17, 20, 45, 45, 59, 73, 102, 112, 143], [2, 15, 18, 34, 37, 38, 39, 47, 56, 58, 61, 82, 120], [46, 80, 84, 97, 101, 110, 111, 111, 121, 129, 140, 146], [0, 12, 14, 18, 18, 24, 24, 30, 33, 45, 87, 117, 145], [3, 20, 56, 70, 70, 89, 109, 112, 119, 141], [40, 70, 82, 115, 126, 145, 147], [0, 16, 23, 74, 76, 89, 99, 111, 123, 123, 141, 141, 145], [31, 35, 35, 36, 66, 72, 96, 101, 107], [25, 36, 37, 47, 71, 97, 102, 117, 127, 132, 139], [4, 24, 26, 32, 37, 39, 44, 92, 104, 135, 143, 146], [1, 13, 16, 20, 33, 35, 76, 82, 86, 99, 104, 116, 142], [15, 82, 103, 105, 113, 136, 138], [15, 26, 52, 66, 83, 111, 111, 125, 147], [22, 35, 38, 63, 63, 67, 67, 134], [6, 12, 18, 28, 55, 58, 67, 126], [35, 54, 60, 66, 92, 135], [12, 14, 32, 36, 45, 56, 75, 81, 99, 122, 130, 133, 149], [1, 6, 27, 34, 60, 62, 92, 101, 101, 117, 123, 125, 148], [6, 14, 15, 18, 21, 35, 86, 89, 109, 121, 127, 128, 139], [21, 22, 38, 40, 68, 70, 89, 98, 104], [10, 41, 76, 93, 117, 122, 125, 132, 133], [1, 23, 57, 88, 113, 117, 118, 147], [6, 14, 22, 41, 64, 84, 95, 98, 103, 129], [5, 8, 59, 76, 91, 102, 108, 115, 131], [8, 20, 50, 84, 110, 116], [12, 19, 63, 68, 92, 101, 111, 125], [1, 20, 57, 69, 83, 85, 119, 135, 140], [1, 7, 9, 19, 45, 52, 57, 64, 107, 132], [8, 9, 10, 43, 53, 55, 66, 81, 82, 85, 140], [1, 26, 39, 47, 70, 87, 88, 100, 106, 143], [4, 37, 87, 92, 97, 119, 148], [39, 70, 75, 94, 124, 136, 145], [21, 24, 29, 31, 32, 59, 73, 90, 103, 116, 127, 132, 143], [5, 12, 29, 34, 103, 104, 129, 136, 146, 147], [0, 31, 39, 44, 48, 52, 77, 86], [52, 53, 61, 114, 133, 136, 142, 148], [21, 30, 52, 63, 84, 86, 100, 105, 114, 114, 126, 141], [34, 36, 60, 85, 97, 110, 150], [1, 3, 74, 82, 87, 99, 122, 133, 140, 140], [0, 1, 23, 28, 100, 112, 119], [13, 37, 51, 52, 57, 58, 65, 91, 122, 124, 139], [4, 6, 7, 11, 12, 14, 57], [28, 29, 66, 76, 77, 79, 84, 92, 126, 143], [32, 38, 64, 78, 95, 101, 103, 107, 110, 110, 139, 140, 141], [9, 18, 21, 62, 79, 98, 115, 117, 129, 133, 149], [47, 55, 75, 81, 110, 121, 143], [51, 54, 65, 97, 129, 142], [5, 17, 28, 29, 65, 66, 94, 131], [0, 13, 14, 16, 54, 69, 71, 77, 98, 98, 100, 108, 139], [18, 23, 23, 36, 40, 56, 64, 90, 98, 139], [1, 34, 38, 48, 56, 65, 102, 108, 118, 119, 126, 134, 136], [1, 10, 13, 23, 36, 61, 62, 122, 123], [13, 17, 26, 26, 30, 52, 58, 62, 69, 79, 84, 89, 135], [1, 42, 73, 75, 118, 131, 135, 142], [36, 36, 52, 62, 77, 80, 82, 88, 92, 105, 111, 128, 144], [10, 28, 51, 78, 125, 125, 144], [44, 87, 110, 120, 126, 127, 136, 145], [12, 58, 69, 77, 81, 84, 91, 93, 116, 136, 139], [1, 4, 54, 86, 110, 127], [2, 8, 16, 82, 99, 112, 140, 142, 146], [21, 41, 50, 117, 122, 125, 144, 146], [5, 7, 25, 44, 49, 70, 109, 121], [11, 21, 48, 100, 107, 126, 130, 142], [4, 42, 52, 91, 95, 98, 111], [40, 48, 66, 82, 101, 106, 145], [2, 15, 20, 39, 62, 66, 68, 79, 84, 109, 126, 134], [12, 24, 25, 40, 43, 55, 57, 117, 126, 130, 140], [16, 79, 91, 98, 109, 109, 110, 111, 111, 114], [9, 12, 52, 55, 64, 67, 94, 99, 100, 100, 102, 111, 129], [10, 23, 38, 46, 99, 106, 108, 125, 140], [2, 11, 14, 14, 29, 39, 77, 102, 110], [1, 35, 52, 85, 94, 95, 113, 113], [1, 16, 17, 18, 19, 20, 23, 27, 40, 56, 71, 85, 118], [4, 16, 48, 54, 102, 120, 136, 137, 146, 146], [16, 53, 53, 70, 79, 80, 90, 124, 130, 131, 133, 134, 141], [31, 67, 69, 94, 114, 129], [13, 20, 28, 57, 60, 70, 98, 142], [2, 32, 55, 90, 118, 136, 138], [1, 19, 23, 69, 80, 96], [53, 67, 77, 85, 91, 97, 100, 107, 133, 137], [6, 13, 30, 42, 71, 119, 120, 148], [15, 22, 32, 61, 61, 98, 143], [19, 26, 29, 61, 95, 100, 109, 120, 146], [16, 18, 30, 66, 67, 76, 82, 89], [45, 60, 62, 72, 79, 97, 108, 118, 120, 132, 137, 141], [30, 58, 101, 128, 140, 142], [9, 45, 72, 81, 86, 95, 117, 124, 133], [23, 36, 45, 62, 75, 81, 91, 93, 107, 118, 123, 137, 140], [1, 8, 39, 48, 62, 63, 64, 113, 127, 131, 144], [24, 36, 54, 66, 85, 145], [3, 47, 79, 91, 115, 119, 131, 143, 144, 148], [7, 20, 37, 69, 77, 78, 135], [10, 10, 28, 54, 64, 77, 102, 116, 124, 149], [3, 33, 72, 75, 81, 121, 126, 130, 137], [5, 41, 47, 108, 110, 139, 150], [38, 73, 75, 88, 90, 99, 101, 119, 142], [43, 55, 65, 70, 73, 77, 122, 132, 132], [0, 13, 28, 31, 43, 54, 58, 67, 86, 149], [3, 8, 28, 34, 35, 70, 83, 116, 125], [1, 48, 54, 77, 111, 118, 122, 124, 129, 129, 140, 140], [13, 22, 30, 36, 42, 46, 59, 81, 110, 123, 124, 143], [9, 9, 11, 31, 35, 35, 39, 46, 118, 123, 123, 131], [16, 35, 72, 84, 86, 94, 97, 127, 132, 134], [1, 2, 13, 24, 44, 49, 76, 94, 96, 105, 111, 131, 138], [28, 47, 52, 69, 77, 80], [9, 18, 41, 50, 58, 59, 84, 113], [1, 5, 31, 35, 45, 48, 128, 143, 150], [57, 62, 66, 74, 78, 121, 122, 133, 136], [9, 23, 34, 44, 55, 62, 81, 83, 90, 115, 124], [8, 42, 105, 109, 123, 139], [12, 28, 29, 53, 116, 125, 131, 134, 135], [76, 78, 94, 109, 132, 136], [3, 5, 26, 33, 43, 52, 70, 91, 94, 131], [18, 23, 34, 54, 75, 96, 116, 127, 128, 141, 141, 145], [35, 49, 72, 93, 101, 111], [16, 48, 98, 99, 110, 133], [19, 35, 41, 42, 75, 92, 110, 139, 147], [13, 26, 42, 42, 75, 93, 94, 104, 107, 113, 130, 139], [4, 44, 61, 69, 74, 77, 78, 93, 97, 106, 136, 142], [1, 9, 18, 54, 62, 74, 77, 78, 84, 88, 91, 125], [18, 30, 31, 71, 90, 95, 104, 125, 141], [61, 63, 102, 111, 131, 135, 139], [19, 48, 69, 73, 88, 116, 145], [13, 27, 47, 56, 65, 72, 81, 87, 91, 112, 124, 139], [21, 28, 66, 71, 136, 140], [1, 21, 35, 35, 37, 39, 42, 43, 45, 95, 102, 129], [8, 8, 8, 9, 22, 38, 70, 75, 81, 96, 98, 123, 125], [2, 3, 3, 19, 63, 79, 81], [21, 31, 55, 65, 114, 116, 127], [12, 13, 59, 114, 128, 148], [12, 23, 38, 39, 69, 103, 120, 131], [2, 26, 37, 42, 68, 81, 87, 88, 92, 99, 107, 127, 144], [5, 29, 39, 40, 67, 83, 83, 122, 128, 143], [3, 18, 33, 52, 58, 92, 102, 104, 127, 130, 130], [5, 8, 61, 67, 101, 104, 114, 130], [11, 39, 40, 48, 61, 70, 126], [35, 37, 51, 78, 93, 97, 104, 124, 140], [14, 63, 113, 116, 145, 148], [2, 37, 59, 68, 95, 97, 127, 128, 134, 145], [1, 36, 44, 78, 88, 107], [16, 29, 40, 96, 115, 136, 145], [32, 38, 62, 67, 69, 69, 84, 136, 150], [2, 6, 19, 20, 32, 33, 57, 60, 63, 71, 108, 117, 121], [51, 67, 67, 76, 88, 99, 99, 105, 125, 126, 131, 146], [42, 64, 83, 121, 125, 136, 144], [4, 11, 11, 60, 63, 69, 77, 78, 79, 136], [35, 36, 51, 104, 129, 145, 146, 147], [62, 69, 71, 84, 99, 100], [22, 23, 37, 55, 74, 112, 113, 128, 141], [5, 45, 67, 71, 80, 104, 132, 135], [2, 8, 31, 32, 41, 55, 101, 122, 137, 140, 140], [2, 10, 12, 18, 18, 20, 45, 51, 100, 108, 130, 130], [26, 32, 37, 37, 56, 76, 80, 104, 111, 132, 133, 134, 147], [0, 11, 52, 92, 101, 126, 131], [19, 21, 36, 57, 94, 107, 108, 111], [22, 33, 92, 97, 103, 104, 105, 144], [29, 39, 49, 74, 116, 126, 136, 147, 147, 148], [23, 34, 36, 37, 40, 45, 64, 65, 76, 77, 79, 132], [7, 13, 21, 74, 87, 115, 124, 128, 130, 133, 136, 141, 145], [31, 47, 82, 101, 110, 123], [16, 56, 61, 66, 90, 111, 125, 141, 146, 146, 148], [2, 5, 12, 13, 29, 32, 66, 82, 119], [11, 30, 32, 44, 51, 77, 91, 112, 119, 139, 142], [17, 25, 39, 51, 51, 61, 72, 81, 88, 96, 115, 139, 143], [0, 5, 16, 34, 45, 63, 86, 86, 118, 130, 143, 146], [40, 59, 60, 70, 86, 94, 98, 100, 107, 120, 141], [5, 11, 20, 36, 44, 49, 60, 81, 95, 98, 99, 109], [18, 48, 49, 93, 103, 118, 123, 131, 138, 150, 150], [19, 42, 55, 77, 85, 99], [18, 40, 50, 54, 54, 70, 83, 110, 116, 124, 130, 144, 150], [11, 33, 58, 64, 106, 117, 125, 134], [27, 31, 34, 39, 55, 60, 61, 62, 82, 88, 103, 149], [0, 53, 58, 66, 70, 80, 96, 112, 138, 139, 142], [22, 26, 33, 65, 81, 130, 137, 138], [5, 12, 46, 58, 75, 77, 129, 146], [23, 57, 66, 75, 102, 110, 116, 122, 122, 140], [40, 52, 65, 66, 81, 102, 114, 118, 127], [33, 37, 55, 107, 114, 123], [37, 65, 68, 70, 89, 94, 97, 110, 126, 143, 146], [18, 23, 26, 32, 41, 57, 64, 68, 77, 96, 104, 145, 146], [36, 65, 80, 80, 97, 107, 143, 146], [10, 21, 34, 57, 109, 112, 119, 120, 142, 148], [4, 8, 9, 22, 44, 51, 53, 55, 93, 99, 125, 136, 144], [26, 30, 44, 80, 86, 89, 108, 114, 136], [16, 16, 49, 49, 99, 135, 144, 150], [16, 30, 31, 48, 74, 97, 146], [4, 21, 26, 32, 54, 72, 97, 104, 118, 120, 139, 147], [17, 19, 69, 71, 81, 99, 111], [3, 13, 15, 50, 58, 67, 73, 114, 132, 138, 138, 145], [6, 11, 30, 33, 45, 52, 54, 63, 73, 123, 126], [12, 33, 53, 60, 84, 89, 100, 103, 111, 116, 127, 142], [8, 12, 24, 31, 36, 56, 65, 85, 87, 117, 121], [20, 32, 43, 51, 59, 85, 88, 100, 128, 142], [36, 52, 90, 94, 107, 133, 138], [7, 14, 33, 43, 65, 69, 92, 144, 145], [29, 32, 44, 54, 130, 130], [2, 77, 82, 99, 113, 148], [16, 24, 48, 59, 59, 61, 73, 85, 88, 89, 101, 122, 145], [21, 125, 125, 127, 133, 138], [2, 2, 13, 31, 38, 40, 41, 67, 69, 81, 109, 149], [28, 47, 84, 90, 92, 104, 105, 105, 110], [2, 11, 22, 26, 30, 46, 49, 65, 89, 111], [8, 28, 42, 78, 79, 86, 98, 100, 100, 106, 118], [29, 59, 72, 75, 89, 123, 127, 136, 137], [1, 24, 25, 27, 28, 28, 36, 48, 123], [1, 1, 39, 40, 44, 61, 61, 80, 108, 110, 131, 140], [8, 22, 45, 75, 105, 118, 122, 130, 148], [2, 10, 27, 30, 42, 50, 56, 64, 72, 86, 104, 140], [15, 21, 45, 68, 106, 114], [8, 16, 27, 57, 72, 76, 81, 96, 123], [37, 60, 66, 68, 86, 93, 105, 111, 114, 132, 134, 145, 147], [6, 10, 41, 59, 117, 126], [29, 41, 81, 101, 114, 121, 142, 149], [2, 10, 21, 47, 97, 126], [35, 47, 49, 65, 87, 90, 96, 128, 137, 145], [17, 47, 54, 55, 61, 68, 94, 96, 114, 116], [10, 19, 34, 65, 73, 73, 88, 114, 130, 144], [7, 9, 34, 36, 36, 55, 88, 122], [0, 15, 39, 70, 133, 145], [4, 12, 18, 33, 40, 60, 69, 141], [14, 26, 30, 32, 44, 77, 78, 90, 106, 115, 136, 136], [3, 7, 16, 20, 49, 79, 87, 90, 95, 98, 114, 119], [25, 42, 52, 53, 53, 78, 86, 104, 110, 147], [20, 33, 35, 40, 63, 74, 100, 103, 109, 145], [1, 2, 28, 36, 39, 69, 73, 101, 108, 145], [26, 43, 61, 71, 111, 118], [0, 19, 28, 49, 63, 77, 86, 91, 113, 135, 144], [46, 50, 99, 111, 131, 131, 145], [32, 32, 46, 52, 73, 75, 87, 96, 118, 133, 138, 148, 149], [13, 30, 35, 49, 57, 62, 73, 96, 149], [43, 49, 62, 87, 90, 112], [20, 38, 59, 62, 96, 98, 103, 138, 141], [1, 4, 65, 97, 137, 145], [38, 41, 46, 81, 82, 86, 89, 94, 97, 111, 114, 115, 122], [2, 39, 54, 60, 64, 106, 107, 113, 122, 146], [17, 29, 42, 51, 66, 92, 98, 103, 141], [3, 77, 100, 121, 124, 141, 149], [0, 5, 17, 20, 36, 49, 60, 130, 142], [1, 9, 21, 23, 62, 67, 101, 121, 125, 138, 147], [2, 5, 15, 26, 37, 42, 48, 49, 54, 80, 98, 124, 131], [9, 14, 15, 33, 34, 40, 56, 66, 72, 90, 136, 141, 146], [1, 37, 39, 45, 46, 83, 93, 97, 134], [23, 67, 80, 89, 108, 109, 127, 128, 129, 129, 132], [6, 16, 42, 109, 132, 134, 139], [29, 63, 66, 70, 97, 104, 109, 130, 141, 148], [2, 13, 17, 22, 33, 46, 70, 84, 88, 93, 94, 113], [2, 9, 23, 32, 37, 63, 94, 104, 107, 108, 118, 136], [3, 9, 49, 60, 73, 117, 122], [13, 14, 44, 58, 63, 74, 89, 98, 105, 118, 119, 135], [18, 21, 55, 71, 90, 103, 112, 116, 121, 136, 145, 147, 149], [12, 13, 15, 29, 34, 41, 46, 51, 79, 126, 141], [2, 14, 17, 23, 55, 94, 134], [16, 22, 47, 60, 70, 106, 109, 125, 128, 143, 145, 150], [5, 7, 80, 86, 98, 98, 105, 109, 119, 122], [24, 39, 43, 49, 76, 100, 102, 117], [19, 24, 59, 76, 92, 97, 117, 129, 142], [66, 75, 75, 79, 95, 100, 114, 133, 143], [6, 42, 44, 60, 91, 139, 144], [3, 29, 39, 57, 68, 74, 80, 145], [0, 5, 6, 28, 52, 58, 72, 84, 91], [0, 12, 13, 49, 89, 91, 104, 110, 134], [4, 7, 8, 29, 35, 50, 69, 72, 92, 97, 106, 107, 120], [26, 39, 54, 65, 66, 79, 84, 88, 88, 88, 90, 121, 150], [2, 11, 16, 32, 33, 33, 34, 38, 66, 106, 114, 121], [27, 34, 53, 65, 96, 103, 108], [5, 54, 63, 72, 107, 119, 122], [17, 25, 91, 95, 148, 149], [1, 15, 72, 83, 110, 134, 146], [55, 62, 81, 82, 84, 85, 87, 100, 129], [6, 18, 27, 35, 73, 86, 92, 93, 94, 146], [15, 17, 22, 23, 47, 52, 54, 70, 72, 78, 85, 129, 146], [23, 35, 106, 114, 128, 145], [5, 16, 23, 31, 38, 66, 67, 71, 85, 119, 144, 148], [8, 50, 69, 84, 112, 140, 141], [16, 31, 43, 46, 47, 57, 64, 124, 133], [2, 5, 69, 73, 90, 92, 98, 112, 132, 138, 142, 142], [0, 2, 4, 5, 10, 10, 22, 71, 77, 98, 114, 140, 147], [8, 26, 28, 31, 45, 76, 97, 113, 127, 135, 136], [1, 54, 92, 115, 133, 145, 145, 150], [2, 15, 18, 21, 33, 37, 53, 101, 105, 109, 126, 127, 144], [12, 36, 37, 39, 69, 75, 91, 95, 96, 113], [14, 32, 44, 49, 74, 86, 86, 87, 97, 104, 108, 123, 129], [23, 34, 42, 59, 63, 88, 96, 97, 102, 104, 108], [8, 10, 15, 20, 25, 40, 45, 47, 108, 126, 126, 127], [22, 90, 105, 123, 126, 134, 136, 145], [10, 24, 24, 49, 56, 82, 94, 102, 102, 120, 123, 147], [10, 41, 44, 45, 48, 52, 92, 96, 111, 127, 129, 132, 135], [7, 8, 18, 21, 42, 43, 70, 97, 100, 120, 123, 147, 147], [16, 25, 43, 67, 92, 123], [14, 18, 31, 40, 54, 66, 76, 81, 87, 106, 122, 138], [12, 27, 37, 42, 49, 54, 57, 60, 110, 114, 124, 141], [14, 23, 37, 41, 42, 47, 54, 64, 73, 103], [56, 72, 83, 85, 85, 86, 99, 109], [23, 40, 71, 82, 89, 104, 105, 123, 123, 146, 147], [79, 82, 94, 99, 114, 123, 141, 144, 149], [11, 15, 61, 64, 74, 79, 94, 95, 100, 102, 110], [18, 72, 87, 104, 106, 114, 116, 117, 144, 146], [23, 53, 76, 79, 86, 93, 100, 110, 113], [12, 42, 53, 99, 118, 120, 134, 138, 147], [12, 20, 39, 45, 59, 61, 74, 77, 130, 136, 143], [20, 33, 54, 65, 87, 150], [6, 8, 37, 83, 112, 115, 119], [39, 52, 64, 81, 135, 141], [8, 8, 15, 23, 43, 56, 67, 67, 104, 105, 121, 123, 124], [20, 22, 42, 55, 82, 93, 101, 110, 113, 117, 119], [21, 23, 26, 62, 66, 78, 88, 90, 101, 101, 112, 116, 146], [43, 59, 61, 84, 120, 142], [31, 32, 48, 73, 77, 93, 96, 126, 134, 147], [4, 33, 75, 120, 122, 122], [2, 4, 28, 29, 42, 75], [5, 14, 15, 75, 75, 119, 121], [7, 21, 25, 34, 68, 76, 87, 87, 103, 123, 124, 126, 127], [0, 1, 28, 34, 36, 51, 52, 96, 97, 107, 111, 121, 131], [21, 33, 48, 80, 91, 116, 150], [0, 4, 17, 20, 23, 38, 43, 82, 89, 96, 115, 130], [31, 58, 63, 73, 88, 93, 99, 101, 140, 142, 146], [6, 33, 50, 61, 63, 78, 118, 120, 125, 139], [9, 29, 42, 61, 68, 69, 73, 85, 85, 99, 124, 143, 145], [17, 21, 22, 79, 125, 138, 148], [5, 13, 14, 46, 104, 129, 131], [25, 30, 45, 50, 55, 56, 62, 79, 91, 95, 120, 141], [6, 22, 63, 85, 93, 101, 111], [6, 24, 31, 58, 59, 67, 70, 82, 103, 127], [44, 68, 70, 92, 100, 116, 136, 141], [12, 56, 75, 75, 119, 126, 130, 143, 144, 147], [3, 21, 22, 30, 39, 59, 99, 142], [4, 45, 71, 106, 107, 133, 136, 147, 150], [12, 22, 73, 82, 120, 127, 133, 146], [2, 2, 10, 17, 21, 23, 49, 102, 108, 115, 128, 135], [12, 14, 35, 60, 98, 102, 116, 131], [13, 66, 69, 70, 72, 143, 143, 147], [5, 13, 24, 61, 72, 78, 86, 87, 90, 105, 116, 124, 141], [22, 46, 53, 106, 109, 128, 150], [28, 41, 41, 62, 72, 108, 131, 136], [5, 26, 40, 78, 79, 123, 123, 124, 125, 132, 139, 149, 150], [27, 34, 44, 60, 87, 111, 127, 150], [23, 28, 58, 59, 60, 88, 95, 106, 123, 126, 142], [18, 40, 48, 61, 96, 102, 102, 129], [15, 18, 23, 29, 49, 68, 68, 79, 85, 97, 109, 124, 124], [24, 36, 95, 95, 97, 122, 125, 130, 133, 146], [7, 9, 10, 33, 100, 109, 110, 120, 144], [4, 26, 28, 66, 81, 82, 130, 140], [0, 15, 35, 48, 50, 59, 78, 110, 120, 127, 148], [22, 26, 42, 43, 64, 69, 70, 94, 111, 127, 137], [61, 117, 121, 124, 134, 148], [9, 20, 37, 72, 73, 87, 103, 109, 109, 112, 119, 141, 145], [0, 2, 40, 49, 77, 79, 120, 143], [1, 10, 18, 39, 119, 145], [1, 100, 101, 116, 139, 142], [10, 16, 20, 46, 62, 63, 92, 107, 150], [34, 36, 50, 113, 116, 130], [0, 8, 13, 23, 26, 45, 53, 65, 121, 137, 142, 149], [10, 37, 37, 40, 44, 51, 60, 64], [16, 54, 59, 64, 90, 90, 92, 109, 136, 140, 149], [29, 34, 37, 91, 93, 101, 107, 119], [18, 70, 73, 75, 82, 90, 93, 100], [0, 25, 34, 37, 62, 114, 116], [6, 25, 32, 48, 51, 100, 149], [16, 21, 34, 36, 45, 76, 134, 136, 138, 139, 143], [16, 25, 34, 128, 140, 147, 149], [5, 27, 46, 54, 65, 70, 83, 90, 94, 100, 121, 128, 150], [1, 8, 43, 45, 62, 76, 111, 115, 125, 126, 129], [21, 49, 96, 99, 100, 117, 132, 135, 148], [17, 24, 24, 32, 70, 89, 99, 123, 150], [45, 52, 66, 67, 102, 104, 125, 125, 137], [12, 22, 36, 90, 108, 128], [1, 4, 33, 67, 82, 83, 97, 110, 130, 141], [2, 4, 25, 34, 92, 93, 95], [6, 14, 30, 42, 50, 69, 81, 97, 98, 123, 133, 143], [14, 22, 66, 72, 96, 127, 133, 141], [35, 58, 59, 71, 80, 80, 114, 120, 122], [22, 29, 35, 50, 54, 69, 102, 103, 133, 148, 150], [28, 32, 38, 40, 45, 45, 62, 80, 108], [0, 1, 8, 17, 19, 24, 40, 62, 76, 85, 87, 114, 131], [27, 31, 33, 37, 38, 52, 77, 119, 125, 141, 143], [13, 44, 46, 70, 74, 85, 101, 103, 109, 146, 149, 150], [6, 45, 70, 80, 86, 132, 139], [2, 21, 36, 77, 108, 113, 131], [18, 18, 45, 65, 69, 76, 90, 128, 145], [1, 2, 17, 25, 44, 57, 71, 81, 90, 122, 132, 136], [6, 13, 20, 22, 32, 42, 46, 68, 95, 142, 146], [2, 11, 51, 78, 80, 87, 149], [33, 45, 53, 66, 70, 75, 92, 94, 123, 128, 140, 148, 148], [4, 12, 25, 47, 62, 70, 82, 86, 98, 138], [12, 14, 22, 34, 58, 66, 77, 79, 104, 125, 135, 136, 139], [6, 15, 45, 54, 60, 64, 98, 111], [9, 38, 40, 45, 48, 51, 59, 66, 117, 118, 139], [8, 25, 29, 50, 63, 79, 94, 146]]
    schedule = []
    for k in range(1000):
        schedule.append(order(n = len(data[k]), items = data[k]))
    return schedule

In [9]:
#fonctions d'un worker
#poser 1 colis dans le buffer ou les mains du suivant si dispo
def drop(worker):
    if worker + 1 == nb_workers : #dernier worker
        #print(str(worker)+"drop1")
        current_state.workers[worker].pos = warehouse
        current_state.workers[worker].order = -1
        pick_order(worker)
    elif current_state.workers[worker + 1].pos > current_state.workers[worker]. zone : #le worker suivant ne bloque pas
        if len(current_state.buffer[worker]) < buffer_size[worker] : #le buffer n'est pas plein, on peut poser le colis
            #print(str(worker)+"drop2")
            current_state.workers[worker].pos = current_state.workers[worker].zone
            current_state.buffer[worker].append(waiting_order(current_state.workers[worker].order, current_state.workers[worker].item))
            current_state.workers[worker].order = -1
            pick_order(worker)
        else : #le buffer est plein
            #print(str(worker)+"drop3")
            current_state.workers[worker].pos = current_state.workers[worker].zone - 1
            for event1 in current_state.events.queue :
                if event1.worker == worker + 1 :
                     current_state.events.add(event(event1.time, worker))  #on attend que le worker suivant fasse un mouvement         
    elif current_state.workers[worker + 1].pos == current_state.workers[worker].pos + 1 : #le worker suivant est disponible
         #print(str(worker)+"drop4")
         if current_state.workers[worker + 1].order == -1 : 
            give(worker)
         else :
              for event1 in current_state.events.queue :
                if event1.worker == worker + 1 :
                     current_state.events.add(event(event1.time, worker))  #on attend que le worker suivant fasse un mouvement
    else : #le worker suivant bloque et est occupé
        #print(str(worker)+"drop5")
        current_state.workers[worker].pos = current_state.workers[worker + 1].pos - 1
        for event1 in current_state.events.queue :
                if event1.worker == worker + 1 :
                     current_state.events.add(event(event1.time, worker))  #on attend que le worker suivant fasse un mouvement 

#donner un colis au suivant
def give(worker) :
    #print(str(worker)+"give")
    current_state.workers[worker + 1].order = current_state.workers[worker].order
    current_state.workers[worker + 1].item = current_state.workers[worker].item
    current_state.workers[worker].order = -1
    pick_order(worker)

#aller chercher un colis (buffer ou se rendre disponible)
def pick_order(worker) :
     global pick_rate
     if worker == 0 : # pas de problème de buffer
        #print(str(worker)+"pick_order1")
        current_state.workers[worker].pos = 0
        current_state.workers[worker].order = pick_rate
        current_state.workers[worker].item = 0
        pick_rate = pick_rate + 1
        #print(pick_rate)
        current_state.events.add(event(time, worker))
     elif len(current_state.buffer[worker -1]) > 0 : # le buffer n'est pas vide
        #print(str(worker)+"pick_order2")
        current_state.workers[worker].pos = current_state.workers[worker - 1].zone
        current_state.workers[worker].order = current_state.buffer[worker - 1][0].order
        current_state.workers[worker].item = current_state.buffer[worker - 1][0].item
        current_state.buffer[worker - 1] = current_state.buffer[worker - 1][1::]
        current_state.events.add(event(time, worker))
     else : # le buffer est vide, le worker va se signaler auprès du précédent
        #print(str(worker)+"pick_order3")
        current_state.workers[worker].pos = current_state.workers[worker - 1].pos + 1
        for event1 in current_state.events.queue :
                if event1.worker == worker - 1 :
                     current_state.events.add(event(event1.time, worker))  #on attend que le worker précédent fasse un mouvement 

#aller chercher un objet
def pick_item(worker):
    global lead_times
    item = current_state.workers[worker].item
    order = current_state.workers[worker].order
    if (worker + 1 < nb_workers) and (schedule[order].item[item]>=current_state.workers[worker + 1].pos): #bloqué par le suivant
        #print(str(worker)+"pick_item1")
        current_state.workers[worker].pos = current_state.workers[worker + 1].pos - 1
        for event1 in current_state.events.queue :
                if event1.worker == worker + 1 :
                     current_state.events.add(event(event1.time, worker))  #on attend que le worker suivant fasse un mouvement 
    else :
         #print(str(worker)+"pick_item2")
         current_state.workers[worker].pos = schedule[order].item[item]
         current_state.workers[worker].item = current_state.workers[worker].item + 1
         tprime = expo(1/speed[worker])
         current_state.events.add(event(time + tprime, worker))
         lead_times[order] += tprime


In [10]:
# définir les zones attribuées aux employés
def setZones (_zones):
    zones = _zones.copy()
    for k in range(nb_workers):
        current_state.workers[k].zone = _zones[k]

In [22]:
#caractéristiques du système
nb_workers = 3 #nb d'employés
warehouse = 150 #nb de cases de la chaîne
buffer_size = [10] * (nb_workers - 1) #taille de chaque buffer
speed = [10] * nb_workers #vitesse moyenne des employés
zones = [0] * nb_workers #zones attribuées aux employés
current_state = state(nb_workers , zones)
time = 0
time_max=8*60*60 #durée d'une journée

start = 6 #taille min commande inclue
end = 13 #taille max commande exclue
schedule = liste_commandes()
lead_times = [0] * len(schedule)

In [23]:

#initialise les paramètres d'état
def initialize_state(zones) : 
    #Initialisation du système 
    global current_state
    current_state = state(nb_workers , zones)
    global time
    time = 0

    #Initialisation des workers
    global pick_rate
    pick_rate = nb_workers
    
    for k in range(nb_workers):
        current_state.workers[k].order = nb_workers - k - 1

#fait tourner la simulation pour l'état en cours
def run():
    #boucle principale
    global time
    global current_state
    global time_max
    global schedule
    global lead_times

    while time<time_max:
        current_event = current_state.events.remove()
        time = current_event.time
        worker = current_event.worker
        worker_state = current_state.workers[worker]

        if worker_state.order == -1 : #le worker est disponible
            pick_order(worker)
        elif worker_state.item >= len (schedule[worker_state.order].item) : #la commande est prête
            #print("alpha")
            drop(worker)
        else : 
            # le worker suivant est diponible
            if (worker + 1 != nb_workers) and (current_state.workers[worker + 1].pos == worker_state.pos + 1) and (current_state.workers[worker + 1].order == -1) :
                give(worker)
            else : #le worker suivant est occupé
                if schedule[worker_state.order].item[worker_state.item]>worker_state.zone : # le prochain objet n'est pas dans la zone du worker
                    #print("beta" + str(schedule[worker_state.order].item[worker_state.item]))
                    drop(worker)
                else :  
                    pick_item(worker)
        #current_state.events.affiche()
        #print(str(worker) + " " + str(worker_state.pos) + " " + str(worker_state.order) + " " + str(worker_state.item), end = "  ||  ")
        #print(schedule[worker_state.order].item, end ='  ||  ')
        #print(str(len(current_state.buffer[0])) + " " + str(len(current_state.buffer[1])))

    return pick_rate, statistics.mean(lead_times)

In [24]:
#choisir les zones
zones = [50, 100, 150]
pick_rate, lead_time = 0, 0
for k in range(10):
   #initialiser les états
   initialize_state(zones)
   #faire tourner la boucle principale
   _pick_rate, _lead_time = run()
   pick_rate += _pick_rate
   lead_time += _lead_time
print(pick_rate/10, lead_time/10)

174.4 452.737184012388


In [19]:
pick_rate, lead_time = 0, 0
print(pick_rate, lead_time)

0 0


Pour tester GIT
